In [3]:
import sys
sys.path.append('../..')
from explainn import tools
from explainn import networks
from explainn import train
from explainn import test
from explainn import interpretation
from explainn import utils
import torch
import os
from torch import nn
from sklearn.metrics import average_precision_score
from sklearn import metrics
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pandas as pd
import numpy as np
from explainn.train.train import train_explainn
from explainn.utils.tools import dna_one_hot
from explainn.models.networks import ExplaiNN


In [5]:

# Load the tsv file
df = pd.read_csv('C:/Users/aslak/Master/Github_first/SCRATCH/AS-TAC/AS-TAC_1000bp.tsv', sep='\t', header=None)


In [9]:
# print the head of the df
print(df.head())
print(f"Shape of the df: {df.shape}")

                0                                                  1   2   3   \
0  1:374401-375400  CTGGTCAGGACGTGACAGTACCCCCCCAAAGGTGCAGACCCCGGAT...   0   0   
1  1:374601-375600  GTGGACTCTGCTCCACCCTCGACGTCGCCCACTTAGGTGGTGCCCC...   0   0   
2  1:385601-386600  GGGCATGTTGAGATAAATGTACTAACCGGGAGGGTTGAATGATGTA...   0   0   
3  1:386201-387200  GATTGGATAACTAGGATGCTGCTGGTAGAGTAGCTAGCTAGCTTAC...   0   0   
4  1:386401-387400  GATGCTGCTGGTAGAGTAGCTAGCTAGCTTCCTAGCTGTACCGACT...   0   0   

   4   5   6   7   8   9   ...  55  56  57  58  59  60  61  62  63  64  
0   0   0   0   0   0   0  ...   1   0   0   0   0   0   0   0   0   0  
1   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
2   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
3   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
4   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  

[5 rows x 65 columns]
Shape of the df: (1638680, 65)


In [17]:
#print how many rows that starts with "21" in the first column
print(f"Number of rows that starts with 21: {df[df[0].str.startswith('21')].shape[0]}")
# print how many rows that starts with "25" in the first column
print(f"Number of rows that starts with 25: {df[df[0].str.startswith('25')].shape[0]}")
# print how many rows that starts with "21" or "25" in the first column
print(f"Number of rows that starts with 21 or 25: {df[df[0].str.startswith('21') | df[0].str.startswith('25')].shape[0]}")
# print what portion of the data that starts with "21" or "25"
print(f"Portion of the data that starts with 21 or 25: {(df[df[0].str.startswith('21')].shape[0] + df[df[0].str.startswith('25')].shape[0])/df.shape[0]}")


Number of rows that starts with 21: 36084
Number of rows that starts with 25: 34250
Number of rows that starts with 21 or 25: 70334
Portion of the data that starts with 21 or 25: 0.042921131642541556


In [33]:

# # make a subset of the data that starts with "21" or "25"
# df_21_25 = df[df[0].str.startswith('21') | df[0].str.startswith('25')]
# # make a subset of the rest of the data
# df_excluded_21_25 = df[~(df[0].str.startswith('21') | df[0].str.startswith('25'))]

# # remove chrom ranges if there are 65 columns
# if df_21_25.shape[1] == 65:
#     df_21_25 = df_21_25.drop(0, axis=1)
# # remove chrom ranges if there are 65 columns
# if df_excluded_21_25.shape[1] == 65:
#     df_excluded_21_25 = df_excluded_21_25.drop(0, axis=1)


# Make a new df_21_25_added from where you add a random 5 % of the rows of df_excluded_21_25 to df_21_25, and remove those rows from df_excluded_21_25 to a new df with _removed. Dataframes does not have attribute called append
sample = df_excluded_21_25.sample(frac=0.05)
df_21_25_10 = pd.concat([df_21_25, sample])
df_excluded_21_25_10 = df_excluded_21_25.drop(sample.index)

In [28]:
#check that the row of the two last dataframes add up to the original
print(f"Shape of df_21_25_added: {df_21_25_10.shape}")
print(f"Shape of df_excluded_21_25_removed: {df_excluded_21_25_10.shape}")
print(f"Shape of df: {df.shape}")
print(f"Shape of df_21_25_added + df_excluded_21_25_removed: {df_21_25_10.shape[0] + df_excluded_21_25_10.shape[0]}")


Shape of df_21_25_added: (148751, 64)
Shape of df_excluded_21_25_removed: (1489929, 64)
Shape of df: (1638680, 65)
Shape of df_21_25_added + df_excluded_21_25_removed: 1638680


In [15]:

# count the number of sequences containing at least one 'N'
count_N = df[1].str.contains('N').sum()

# count the number of sequences not of length 1000
count_not_1000 = (df[1].str.len() < 1000).sum()
good_lines = (df[1].str.len() == 1000) & (~df[1].str.contains('N'))
print(f"Number of sequences that contain at least one 'N': {count_N}")
print(f"Number of sequences that are not of length 1000: {count_not_1000}")
print(f"Number of sequences that are good: {good_lines.sum()}")
print(f"Fraction of sequences that are good: {good_lines.sum()/df.shape[0]}")

Number of sequences that contain at least one 'N': 0
Number of sequences that are not of length 1000: 0
Number of sequences that are good: 1638680
Fraction of sequences that are good: 1.0


In [5]:
#print the sequences that does not contain an 'A'
print(df[1][~df[1].str.contains('A')])

3981       TCTCTGTCTGTCTCGCTCTCTGTCTGTCTGTCTCGCTCTCTGTCTG...
286411     TCTCTCTCTCTGTGTCTCTCTCTCTCTGTGTCTCTGTGTGTGTCTC...
747627     CTGTCTCTGTTTCTCTGTCTGTCTCTGTCTGTCTCTGTTTCTCTGT...
1525743    TCGCTCTGCGTGTGGCGCTCTCTCGCTCTGCGTGTGGCGCTCTCGC...
1525744    CTCTGCGTGCGTGTGGCGCTCTCTCTCTCGCTCTGCGTGCGTGTGG...
                                 ...                        
1627339    TGTGTGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGTGTGTGTGTGT...
1627340    GTGTGTGTGTTGTCGGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGT...
1627341    GGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGTGTGTGTGTGTGTTG...
1631206    TCTCTCTCTGTGTCTCTCTCTCTCTCTGTGTGTCTCTCTCTCTGTC...
1631231    CTCTCTGTGTGTGTGTGTGTCTGTCTGTCTCTCTCTCTGTGTGTGT...
Name: 1, Length: 77, dtype: object


In [6]:
# print rownames of test_feat
print(df[0])


0                          1:374401-375400
1                          1:374601-375600
2                          1:385601-386600
3                          1:386201-387200
4                          1:386401-387400
                        ...               
1638675    CAJNNT020004222.1:229401-230400
1638676    CAJNNT020004222.1:229601-230600
1638677    CAJNNT020004222.1:229801-230800
1638678    CAJNNT020004222.1:230001-231000
1638679    CAJNNT020004222.1:230201-231200
Name: 0, Length: 1638680, dtype: object


In [ ]:
sq

In [7]:
# Let's make a .h5
import pandas as pd
import numpy as np
import h5py
from sklearn.model_selection import train_test_split

# make a subset of df with only 50 000 random rows
df = df.sample(n=50000, random_state=42)

# load target_labels
with open("../../dna-sequence-models/downloads/bed_list.txt", 'r') as f:
    labels = f.read().splitlines()

# Remove "AS-TAC-peaks/AtlanticSalmon_ATAC_" and ".mLb.clN_peaks.narrowPeak" from the strings in labels list
labels = [label.replace("AS-TAC-peaks/AtlanticSalmon_ATAC_", "").replace(".mLb.clN_peaks.narrowPeak", "") for label in labels]

# remove chrom ranges if chrom column is there, i.e. if column 1 has a ":" in it
if df[0].str.contains(":").sum() > 0:
    df = df.drop(0, axis=1)
    
# separate sequences and binary features
seqs = df.iloc[:, 0]
features = df.iloc[:, 1:]

# one hot encode sequences
seqs_one_hot = np.array([dna_one_hot(str(seq)) for seq in seqs])
seqs = seqs.apply(lambda x: pd.Series(list(x)))

# split data into train, test, valid
train_seq, test_seq, train_feat, test_feat = train_test_split(seqs_one_hot, features, test_size=0.20, random_state=42)
train_seq, valid_seq, train_feat, valid_feat = train_test_split(train_seq, train_feat, test_size=0.25, random_state=42)


In [1]:
########### MAKE A 5 % SPLIT WITH 21 25 SEPARATED ###########
df_21_25 = df[df[0].str.startswith('21') | df[0].str.startswith('25')]
# make a subset of the rest of the data
df_excluded_21_25 = df[~(df[0].str.startswith('21') | df[0].str.startswith('25'))]

# remove chrom ranges if there are 65 columns
if df_21_25.shape[1] == 65:
    df_21_25 = df_21_25.drop(0, axis=1)
# remove chrom ranges if there are 65 columns
if df_excluded_21_25.shape[1] == 65:
    df_excluded_21_25 = df_excluded_21_25.drop(0, axis=1)

# separate sequences and binary features
seqs_21_25 = df_21_25.iloc[:, 0]
features_21_25 = df_21_25.iloc[:, 1:]
seqs_excluded_21_25 = df_excluded_21_25.iloc[:, 0]
features_excluded_21_25 = df_excluded_21_25.iloc[:, 1:]

# one hot encode sequences
seqs_21_25 = np.array([dna_one_hot(str(seq)) for seq in seqs_21_25])
seqs_excluded_21_25 = np.array([dna_one_hot(str(seq)) for seq in seqs_excluded_21_25])

# split data into train_21, test_21
train_seq_21, test_seq_21, train_feat_21, test_feat_21 = train_test_split(seqs_excluded_21_25, features_excluded_21_25, test_size=0.05, random_state=42)
valid_seq_21, valid_feat_21 = seqs_21_25, features_21_25

# create .h5 file
with h5py.File(args['output'] + "AS-TAC_21_25_05.h5", 'w') as hf:
    hf.create_dataset('train_in', data=train_seq_21)
    hf.create_dataset('valid_in', data=valid_seq_21)
    hf.create_dataset('test_in', data=test_seq_21)
    hf.create_dataset('train_out', data=train_feat_21)
    hf.create_dataset('valid_out', data=valid_feat_21)
    hf.create_dataset('test_out', data=test_feat_21)
    hf.create_dataset('target_labels', data=labels)

NameError: name 'df' is not defined

In [9]:

# create .h5 file
with h5py.File('../../../SCRATCH/AS-TAC/AS-TAC_50K.h5', 'w') as hf:
    hf.create_dataset('train_in', data=train_seq)
    hf.create_dataset('valid_in', data=valid_seq)
    hf.create_dataset('test_in', data=test_seq)
    hf.create_dataset('train_out', data=train_feat)
    hf.create_dataset('valid_out', data=valid_feat)
    hf.create_dataset('test_out', data=test_feat)
    hf.create_dataset('target_labels', data=labels)

In [10]:
labels

['Liver_Mature_Male_R1',
 'MidSomitogenesis_R1',
 'Muscle_Immature_Female_R3',
 'Muscle_Mature_Male_R3',
 'LateSomitogenesis_R3',
 'Liver_Immature_Female_R1',
 'Gonad_Immature_Female_R3',
 'MidGastrulation_R3',
 'MidGastrulation_R1',
 'Muscle_Mature_Female_R3',
 'LateSomitogenesis_R1',
 'Brain_Immature_Male_R2',
 'Gonad_Immature_Male_R1',
 'Liver_Immature_Female_R2',
 'LateBlastulation_R3',
 'EarlySomitogenesis_R1',
 'MidSomitogenesis_R3',
 'Liver_Mature_Female_R3',
 'Brain_Mature_Male_R1',
 'Liver_Immature_Male_R3',
 'Liver_Mature_Female_R2',
 'Brain_Immature_Male_R3',
 'Brain_Immature_Female_R3',
 'Gonad_Immature_Female_R2',
 'LateSomitogenesis_R2',
 'Brain_Mature_Female_R2',
 'Gonad_Mature_Female_R3',
 'Brain_Mature_Male_R2',
 'EarlySomitogenesis_R3',
 'Brain_Immature_Male_R1',
 'Liver_Mature_Male_R3',
 'Muscle_Mature_Female_R2',
 'Muscle_Immature_Female_R1',
 'Brain_Immature_Female_R2',
 'Brain_Mature_Male_R3',
 'Liver_Immature_Male_R1',
 'Muscle_Immature_Female_R2',
 'MidGastrulat